In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import random
from joblib import parallel_backend
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
import gc


In [ ]:
import pandas
import os

# This query represents dataset "Eczema" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_57765325_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (133835)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_57765325_person_df = pandas.read_gbq(
    dataset_57765325_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_57765325_person_df.head(5)

In [ ]:
import pandas
import os

# This query represents dataset "Eczema" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_88415271_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (1586134, 1585855, 1585710, 43528895, 40192389, 1740639)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (133835)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) )
        )"""

dataset_88415271_survey_df = pandas.read_gbq(
    dataset_88415271_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_88415271_survey_df.head(5)

In [ ]:
eczema_survey_processed = dataset_88415271_survey_df.groupby(['person_id', 'survey'])['answer'].agg('max').reset_index()
eczema_survey_processed = eczema_survey_processed.pivot(index='person_id', columns='survey', values='answer')
eczema_survey_processed = eczema_survey_processed.reset_index()
eczema_survey_processed.fillna('unknown', inplace=True)

del dataset_88415271_survey_df
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset "Eczema" for domain "fitbit_heart_rate_summary" and was generated for All of Us Controlled Tier Dataset v7
dataset_57765325_fitbit_heart_rate_summary_sql = """
    SELECT
        heart_rate_summary.person_id,
        heart_rate_summary.date,
        heart_rate_summary.zone_name,
        heart_rate_summary.min_heart_rate,
        heart_rate_summary.max_heart_rate,
        heart_rate_summary.minute_in_zone,
        heart_rate_summary.calorie_count 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.heart_rate_summary` heart_rate_summary   
    WHERE
        heart_rate_summary.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (133835)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_57765325_fitbit_heart_rate_summary_df = pandas.read_gbq(
    dataset_57765325_fitbit_heart_rate_summary_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_57765325_fitbit_heart_rate_summary_df.head(5)

In [ ]:
fitbit_heart_rate = (
    dataset_57765325_fitbit_heart_rate_summary_df.groupby('person_id')
    .agg(
        mean_min_heart_rate = ('min_heart_rate', 'mean'),
        mean_max_heart_rate = ('max_heart_rate', 'mean'),
        mean_minute_in_zone=('minute_in_zone', 'mean'),
        max_minute_in_zone=('minute_in_zone', 'max'),
        heart_rate_count=('calorie_count', 'count'),
        calorie_mean=('calorie_count', 'mean'),
        calorie_max=('calorie_count', 'max'),
    )
    .reset_index()
)
fitbit_heart_rate

del dataset_57765325_fitbit_heart_rate_summary_df
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset "Eczema" for domain "fitbit_heart_rate_level" and was generated for All of Us Controlled Tier Dataset v7
dataset_57765325_fitbit_heart_rate_level_sql = """
    SELECT
        heart_rate_minute_level.person_id,
        CAST(heart_rate_minute_level.datetime AS DATE) as date,
        AVG(heart_rate_value) avg_rate 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.heart_rate_minute_level` heart_rate_minute_level   
    WHERE
        heart_rate_minute_level.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (133835)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) ) 
    GROUP BY
        person_id,
        date"""

dataset_57765325_fitbit_heart_rate_level_df = pandas.read_gbq(
    dataset_57765325_fitbit_heart_rate_level_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_57765325_fitbit_heart_rate_level_df.head(5)

In [ ]:
fitbit_avg_rate = (
    dataset_57765325_fitbit_heart_rate_level_df.groupby('person_id')
    .agg(
        mean_avg_rate = ('avg_rate', 'mean'),
        max_avg_rate = ('avg_rate', 'max'),
        min_avg_rate=('avg_rate', 'min'),
    )
    .reset_index()
)
fitbit_avg_rate

del dataset_57765325_fitbit_heart_rate_level_df
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset "Eczema" for domain "fitbit_intraday_steps" and was generated for All of Us Controlled Tier Dataset v7
dataset_57765325_fitbit_intraday_steps_sql = """
    SELECT
        steps_intraday.person_id,
        CAST(steps_intraday.datetime AS DATE) as date,
        SUM(CAST(steps_intraday.steps AS INT64)) as sum_steps 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.steps_intraday` steps_intraday   
    WHERE
        steps_intraday.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (133835)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) ) 
    GROUP BY
        person_id,
        date"""

dataset_57765325_fitbit_intraday_steps_df = pandas.read_gbq(
    dataset_57765325_fitbit_intraday_steps_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_57765325_fitbit_intraday_steps_df.head(5)

In [ ]:
fitbit_sum_steps = (
    dataset_57765325_fitbit_intraday_steps_df.groupby('person_id')
    .agg(
        mean_avg_steps = ('sum_steps', 'mean'),
        max_avg_steps = ('sum_steps', 'max'),
        min_avg_steps=('sum_steps', 'min'),
    )
    .reset_index()
)
fitbit_sum_steps

del dataset_57765325_fitbit_intraday_steps_df
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset "Eczema" for domain "fitbit_activity" and was generated for All of Us Controlled Tier Dataset v7
dataset_57765325_fitbit_activity_sql = """
    SELECT
        activity_summary.person_id,
        activity_summary.date,
        activity_summary.activity_calories,
        activity_summary.calories_bmr,
        activity_summary.calories_out,
        activity_summary.elevation,
        activity_summary.fairly_active_minutes,
        activity_summary.floors,
        activity_summary.lightly_active_minutes,
        activity_summary.marginal_calories,
        activity_summary.sedentary_minutes,
        activity_summary.steps,
        activity_summary.very_active_minutes 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.activity_summary` activity_summary   
    WHERE
        activity_summary.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (133835)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_57765325_fitbit_activity_df = pandas.read_gbq(
    dataset_57765325_fitbit_activity_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_57765325_fitbit_activity_df.head(5)

In [ ]:
fitbit_activity = (
    dataset_57765325_fitbit_activity_df.groupby('person_id')
    .agg(
        mean_activity_calories = ('activity_calories', 'mean'),
        max_activity_calories = ('activity_calories', 'max'),
        min_activity_calories=('activity_calories', 'min'),
        mean_fairly_active_minutes = ('fairly_active_minutes', 'mean'),
        max_fairly_active_minutes = ('fairly_active_minutes', 'max'),
        min_fairly_active_minutes=('fairly_active_minutes', 'min'),
        mean_very_active_minutes = ('very_active_minutes', 'mean'),
        max_very_active_minutes = ('very_active_minutes', 'max'),
        min_very_active_minutes=('very_active_minutes', 'min'),
        mean_steps = ('steps', 'mean'),
        max_steps = ('steps', 'max'),
        min_steps=('steps', 'min'),
    )
    .reset_index()
)
fitbit_activity


del dataset_57765325_fitbit_activity_df
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset "Eczema" for domain "fitbit_sleep_daily_summary" and was generated for All of Us Controlled Tier Dataset v7
dataset_57765325_fitbit_sleep_daily_summary_sql = """
    SELECT
        sleep_daily_summary.person_id,
        sleep_daily_summary.sleep_date,
        sleep_daily_summary.is_main_sleep,
        sleep_daily_summary.minute_in_bed,
        sleep_daily_summary.minute_asleep,
        sleep_daily_summary.minute_after_wakeup,
        sleep_daily_summary.minute_awake,
        sleep_daily_summary.minute_restless,
        sleep_daily_summary.minute_deep,
        sleep_daily_summary.minute_light,
        sleep_daily_summary.minute_rem,
        sleep_daily_summary.minute_wake 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.sleep_daily_summary` sleep_daily_summary   
    WHERE
        PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (133835)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_57765325_fitbit_sleep_daily_summary_df = pandas.read_gbq(
    dataset_57765325_fitbit_sleep_daily_summary_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_57765325_fitbit_sleep_daily_summary_df.head(5)

In [ ]:
fitbit_sleep = (
    dataset_57765325_fitbit_sleep_daily_summary_df.groupby('person_id')
    .agg(
        mean_minute_asleep = ('minute_asleep', 'mean'),
        max_minute_asleep = ('minute_asleep', 'max'),
        min_minute_asleep=('minute_asleep', 'min'),
        mean_minute_deep = ('minute_deep', 'mean'),
        max_minute_deep = ('minute_deep', 'max'),
        min_minute_deep=('minute_deep', 'min'),
        mean_minute_rem = ('minute_rem', 'mean'),
        max_minute_rem = ('minute_rem', 'max'),
        min_minute_rem=('minute_rem', 'min'),
        mean_minute_light = ('minute_light', 'mean'),
        max_minute_light = ('minute_light', 'max'),
        min_minute_light=('minute_light', 'min'),
    )
    .reset_index()
)
fitbit_sleep

del dataset_57765325_fitbit_sleep_daily_summary_df
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset "Eczema" for domain "zip_code_socioeconomic" and was generated for All of Us Controlled Tier Dataset v7
dataset_57765325_zip_code_socioeconomic_sql = """
    SELECT
        observation.person_id,
        observation.observation_datetime,
        zip_code.zip3_as_string as zip_code,
        zip_code.fraction_assisted_income as assisted_income,
        zip_code.fraction_high_school_edu as high_school_education,
        zip_code.median_income,
        zip_code.fraction_no_health_ins as no_health_insurance,
        zip_code.fraction_poverty as poverty,
        zip_code.fraction_vacant_housing as vacant_housing,
        zip_code.deprivation_index,
        zip_code.acs as american_community_survey_year 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.zip3_ses_map` zip_code 
    JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
            ON CAST(SUBSTR(observation.value_as_string, 0, STRPOS(observation.value_as_string, '*') - 1) AS INT64) = zip_code.zip3  
    WHERE
        observation.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (133835)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) ) 
        AND observation_source_concept_id = 1585250 
        AND observation.value_as_string NOT LIKE 'Res%'"""

dataset_57765325_zip_code_socioeconomic_df = pandas.read_gbq(
    dataset_57765325_zip_code_socioeconomic_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_57765325_zip_code_socioeconomic_df.head(5)

In [ ]:
eczema_merged = pd.merge(dataset_57765325_person_df, dataset_57765325_zip_code_socioeconomic_df, on='person_id', how='inner')
eczema_merged = pd.merge(eczema_merged, fitbit_heart_rate, on='person_id', how='left')
eczema_merged[fitbit_heart_rate.columns] = eczema_merged[fitbit_heart_rate.columns].fillna(-1)
eczema_merged = pd.merge(eczema_merged, fitbit_avg_rate, on='person_id', how='left')
eczema_merged[fitbit_avg_rate.columns] = eczema_merged[fitbit_avg_rate.columns].fillna(-1)
eczema_merged = pd.merge(eczema_merged, fitbit_sum_steps, on='person_id', how='left')
eczema_merged[fitbit_sum_steps.columns] = eczema_merged[fitbit_sum_steps.columns].fillna(-1)
eczema_merged = pd.merge(eczema_merged, fitbit_activity, on='person_id', how='left')
eczema_merged[fitbit_activity.columns] = eczema_merged[fitbit_activity.columns].fillna(-1)
eczema_merged = pd.merge(eczema_merged, fitbit_sleep, on='person_id', how='left')
eczema_merged[fitbit_sleep.columns] = eczema_merged[fitbit_sleep.columns].fillna(-1)
eczema_merged = pd.merge(eczema_merged, eczema_survey_processed, on='person_id', how='left')
eczema_merged[eczema_survey_processed.columns] = eczema_merged[eczema_survey_processed.columns].fillna('unknown')
eczema_merged

In [ ]:
del dataset_57765325_person_df,eczema_survey_processed,
fitbit_heart_rate, fitbit_sum_steps, fitbit_activity, fitbit_sleep, dataset_57765325_zip_code_socioeconomic_df
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_66329268_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id"""

dataset_66329268_person_df = pandas.read_gbq(
    dataset_66329268_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_66329268_person_df.head(5)
import pandas
import os

# This query represents dataset for domain "zip_code_socioeconomic" and was generated for All of Us Controlled Tier Dataset v7
dataset_66329268_zip_code_socioeconomic_sql = """
    SELECT
        observation.person_id,
        observation.observation_datetime,
        zip_code.zip3_as_string as zip_code,
        zip_code.fraction_assisted_income as assisted_income,
        zip_code.fraction_high_school_edu as high_school_education,
        zip_code.median_income,
        zip_code.fraction_no_health_ins as no_health_insurance,
        zip_code.fraction_poverty as poverty,
        zip_code.fraction_vacant_housing as vacant_housing,
        zip_code.deprivation_index,
        zip_code.acs as american_community_survey_year 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.zip3_ses_map` zip_code 
    JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
            ON CAST(SUBSTR(observation.value_as_string, 0, STRPOS(observation.value_as_string, '*') - 1) AS INT64) = zip_code.zip3 
            AND observation_source_concept_id = 1585250 
            AND observation.value_as_string NOT LIKE 'Res%'"""

dataset_66329268_zip_code_socioeconomic_df = pandas.read_gbq(
    dataset_66329268_zip_code_socioeconomic_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_66329268_zip_code_socioeconomic_df.head(5)

In [ ]:
# This query represents dataset for domain "fitbit_heart_rate_summary" and was generated for All of Us Controlled Tier Dataset v7
dataset_29406193_fitbit_heart_rate_summary_sql = """
    SELECT
        heart_rate_summary.person_id,
        heart_rate_summary.date,
        heart_rate_summary.zone_name,
        heart_rate_summary.min_heart_rate,
        heart_rate_summary.max_heart_rate,
        heart_rate_summary.minute_in_zone,
        heart_rate_summary.calorie_count 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.heart_rate_summary` heart_rate_summary """

dataset_29406193_fitbit_heart_rate_summary_df = pandas.read_gbq(
    dataset_29406193_fitbit_heart_rate_summary_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_29406193_fitbit_heart_rate_summary_df.head(5)

In [ ]:
fitbit_heart_rate_all = (
    dataset_29406193_fitbit_heart_rate_summary_df.groupby('person_id')
    .agg(
        mean_min_heart_rate = ('min_heart_rate', 'mean'),
        mean_max_heart_rate = ('max_heart_rate', 'mean'),
        mean_minute_in_zone=('minute_in_zone', 'mean'),
        max_minute_in_zone=('minute_in_zone', 'max'),
        heart_rate_count=('calorie_count', 'count'),
        calorie_mean=('calorie_count', 'mean'),
        calorie_max=('calorie_count', 'max'),
    )
    .reset_index()
)

del dataset_29406193_fitbit_heart_rate_summary_df
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset for domain "fitbit_heart_rate_level" and was generated for All of Us Controlled Tier Dataset v7
dataset_29406193_fitbit_heart_rate_level_sql = """
    SELECT
        heart_rate_minute_level.person_id,
        CAST(heart_rate_minute_level.datetime AS DATE) as date,
        AVG(heart_rate_value) avg_rate 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.heart_rate_minute_level` heart_rate_minute_level  
    GROUP BY
        person_id,
        date"""

dataset_29406193_fitbit_heart_rate_level_df = pandas.read_gbq(
    dataset_29406193_fitbit_heart_rate_level_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_29406193_fitbit_heart_rate_level_df.head(5)

In [ ]:
fitbit_avg_rate_all = (
    dataset_29406193_fitbit_heart_rate_level_df.groupby('person_id')
    .agg(
        mean_avg_rate = ('avg_rate', 'mean'),
        max_avg_rate = ('avg_rate', 'max'),
        min_avg_rate=('avg_rate', 'min'),
    )
    .reset_index()
)

del dataset_29406193_fitbit_heart_rate_level_df
gc.collect()

In [ ]:
# This query represents dataset for domain "fitbit_activity" and was generated for All of Us Controlled Tier Dataset v7
dataset_29406193_fitbit_activity_sql = """
    SELECT
        activity_summary.person_id,
        activity_summary.date,
        activity_summary.activity_calories,
        activity_summary.calories_bmr,
        activity_summary.calories_out,
        activity_summary.elevation,
        activity_summary.fairly_active_minutes,
        activity_summary.floors,
        activity_summary.lightly_active_minutes,
        activity_summary.marginal_calories,
        activity_summary.sedentary_minutes,
        activity_summary.steps,
        activity_summary.very_active_minutes 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.activity_summary` activity_summary """

dataset_29406193_fitbit_activity_df = pandas.read_gbq(
    dataset_29406193_fitbit_activity_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_29406193_fitbit_activity_df.head(5)

In [ ]:
fitbit_activity_all = (
    dataset_29406193_fitbit_activity_df.groupby('person_id')
    .agg(
        mean_activity_calories = ('activity_calories', 'mean'),
        max_activity_calories = ('activity_calories', 'max'),
        min_activity_calories=('activity_calories', 'min'),
        mean_fairly_active_minutes = ('fairly_active_minutes', 'mean'),
        max_fairly_active_minutes = ('fairly_active_minutes', 'max'),
        min_fairly_active_minutes=('fairly_active_minutes', 'min'),
        mean_very_active_minutes = ('very_active_minutes', 'mean'),
        max_very_active_minutes = ('very_active_minutes', 'max'),
        min_very_active_minutes=('very_active_minutes', 'min'),
        mean_steps = ('steps', 'mean'),
        max_steps = ('steps', 'max'),
        min_steps=('steps', 'min'),
    )
    .reset_index()
)

del dataset_29406193_fitbit_activity_df
gc.collect()

In [ ]:
dataset_29406193_fitbit_intraday_steps_sql = """
    SELECT
        steps_intraday.person_id,
        CAST(steps_intraday.datetime AS DATE) as date,
        SUM(CAST(steps_intraday.steps AS INT64)) as sum_steps 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.steps_intraday` steps_intraday  
    GROUP BY
        person_id,
        date"""

dataset_29406193_fitbit_intraday_steps_df = pandas.read_gbq(
    dataset_29406193_fitbit_intraday_steps_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_29406193_fitbit_intraday_steps_df.head(5)

In [ ]:
fitbit_sum_steps_all = (
    dataset_29406193_fitbit_intraday_steps_df.groupby('person_id')
    .agg(
        mean_avg_steps = ('sum_steps', 'mean'),
        max_avg_steps = ('sum_steps', 'max'),
        min_avg_steps=('sum_steps', 'min'),
    )
    .reset_index()
)

del dataset_29406193_fitbit_intraday_steps_df
gc.collect()

In [ ]:
# This query represents dataset for domain "fitbit_sleep_daily_summary" and was generated for All of Us Controlled Tier Dataset v7
dataset_29406193_fitbit_sleep_daily_summary_sql = """
    SELECT
        sleep_daily_summary.person_id,
        sleep_daily_summary.sleep_date,
        sleep_daily_summary.is_main_sleep,
        sleep_daily_summary.minute_in_bed,
        sleep_daily_summary.minute_asleep,
        sleep_daily_summary.minute_after_wakeup,
        sleep_daily_summary.minute_awake,
        sleep_daily_summary.minute_restless,
        sleep_daily_summary.minute_deep,
        sleep_daily_summary.minute_light,
        sleep_daily_summary.minute_rem,
        sleep_daily_summary.minute_wake 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.sleep_daily_summary` sleep_daily_summary """

dataset_29406193_fitbit_sleep_daily_summary_df = pandas.read_gbq(
    dataset_29406193_fitbit_sleep_daily_summary_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_29406193_fitbit_sleep_daily_summary_df.head(5)

In [ ]:
fitbit_sleep_all = (
    dataset_29406193_fitbit_sleep_daily_summary_df.groupby('person_id')
    .agg(
        mean_minute_asleep = ('minute_asleep', 'mean'),
        max_minute_asleep = ('minute_asleep', 'max'),
        min_minute_asleep=('minute_asleep', 'min'),
        mean_minute_deep = ('minute_deep', 'mean'),
        max_minute_deep = ('minute_deep', 'max'),
        min_minute_deep=('minute_deep', 'min'),
        mean_minute_rem = ('minute_rem', 'mean'),
        max_minute_rem = ('minute_rem', 'max'),
        min_minute_rem=('minute_rem', 'min'),
        mean_minute_light = ('minute_light', 'mean'),
        max_minute_light = ('minute_light', 'max'),
        min_minute_light=('minute_light', 'min'),
    )
    .reset_index()
)

del dataset_29406193_fitbit_sleep_daily_summary_df
gc.collect()

In [ ]:
all_merged = pd.merge(dataset_66329268_person_df, dataset_66329268_zip_code_socioeconomic_df, on='person_id', how='inner')
all_merged = pd.merge(all_merged, fitbit_heart_rate_all, on='person_id', how='left')
all_merged[fitbit_heart_rate_all.columns] = all_merged[fitbit_heart_rate_all.columns].fillna(-1)
all_merged = pd.merge(all_merged, fitbit_avg_rate_all, on='person_id', how='left')
all_merged[fitbit_avg_rate_all.columns] = all_merged[fitbit_avg_rate_all.columns].fillna(-1)
all_merged = pd.merge(all_merged, fitbit_sum_steps_all, on='person_id', how='left')
all_merged[fitbit_sum_steps_all.columns] = all_merged[fitbit_sum_steps_all.columns].fillna(-1)
all_merged = pd.merge(all_merged, fitbit_activity_all, on='person_id', how='left')
all_merged[fitbit_activity_all.columns] = all_merged[fitbit_activity_all.columns].fillna(-1)
all_merged = pd.merge(all_merged, fitbit_sleep_all, on='person_id', how='left')
all_merged[fitbit_sleep_all.columns] = all_merged[fitbit_sleep_all.columns].fillna(-1)

In [ ]:
del dataset_66329268_person_df, dataset_66329268_zip_code_socioeconomic_df, fitbit_heart_rate_all, fitbit_avg_rate_all, 
fitbit_sum_steps_all, fitbit_activity_all, fitbit_sleep_all
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_88696117_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (1586134, 1585855, 1585710)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  """ 

dataset_88696117_survey_df = pandas.read_gbq(
    dataset_88696117_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_88696117_survey_df.head(5)

In [ ]:
import pandas
import os

# This query represents dataset for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_88696117_survey_sql_2 = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (43528895)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  """ 

dataset_88696117_survey_df_2 = pandas.read_gbq(
    dataset_88696117_survey_sql_2,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_88696117_survey_df_2.head(5)

In [ ]:
import pandas as pd
compiled_all_1 = pd.concat([dataset_88696117_survey_df, dataset_88696117_survey_df_2], ignore_index=True)

del dataset_88696117_survey_df, dataset_88696117_survey_df_2
gc.collect()

In [ ]:
all_survey_processed_1 = compiled_all_1.groupby(['person_id', 'survey'])['answer'].agg('max').reset_index()

del compiled_all_1
gc.collect()

In [ ]:
import pandas
import os

# This query represents dataset for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_88696117_survey_sql_3 = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (40192389)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  """ 

dataset_88696117_survey_df_3 = pandas.read_gbq(
    dataset_88696117_survey_sql_3,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_88696117_survey_df_3.head(5)

In [ ]:
import pandas
import os

# This query represents dataset for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_88696117_survey_sql_4 = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (1740639)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  """ 

dataset_88696117_survey_df_4 = pandas.read_gbq(
    dataset_88696117_survey_sql_3,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_88696117_survey_df_4.head(5)

In [ ]:
import pandas as pd

In [ ]:
compiled_all_2 = pd.concat([dataset_88696117_survey_df_3, dataset_88696117_survey_df_4], ignore_index=True)

del dataset_88696117_survey_df_3, dataset_88696117_survey_df_4
gc.collect()

In [ ]:
all_survey_processed_2 = compiled_all_2.groupby(['person_id', 'survey'])['answer'].agg('max').reset_index()

del compiled_all_2
gc.collect()

In [ ]:
all_survey_processed = pd.concat([all_survey_processed_1, all_survey_processed_2], ignore_index=True)
del all_survey_processed_1, all_survey_processed_2
gc.collect()

In [ ]:
all_survey_processed = all_survey_processed.pivot(index='person_id', columns='survey', values='answer')
all_survey_processed = all_survey_processed.reset_index()
all_survey_processed.fillna('unknown', inplace=True)

In [ ]:
all_survey_processed

In [ ]:
eczema_rows = all_survey_processed[all_survey_processed['Lifestyle'].str.contains('Opioids', case=False, na=False)]
eczema_rows

In [ ]:
all_merged = pd.merge(all_merged, all_survey_processed, on='person_id', how='left')
all_merged[all_survey_processed.columns] = all_merged[all_survey_processed.columns].fillna('unknown')
del all_survey_processed
gc.collect()

In [ ]:
eczema_merged['Eczema'] = 1
all_merged['Eczema'] = 0
combined_df = pd.concat([eczema_merged, all_merged])
final_df = combined_df.sort_values('Eczema', ascending=False).drop_duplicates(subset=['person_id'])
final_df.reset_index(drop=True, inplace=True)
# zipcode = final_df['zip_code'].tolist()
# final_df.loc[:,'zip_code'] = [int(zc.replace("**", "")) for zc in zipcode]
# final_df['zip_code'] = pd.to_numeric(final_df['zip_code'], errors='coerce')
final_df.dtypes

In [ ]:
white_only = final_df[final_df['race'] == 'White']
white_only

In [ ]:
white_only.loc[:, 'year_of_birth'] = white_only['date_of_birth'].dt.year
white_only = white_only.drop(['person_id', 'race', 'race_concept_id', 'gender_concept_id', 'ethnicity_concept_id','sex_at_birth_concept_id', 'observation_datetime', 'date_of_birth'], axis = 1)
cat_columns = white_only.select_dtypes(include=['object', 'category']).columns
whites_encoded = pd.get_dummies(white_only, columns=cat_columns, drop_first=True)
whites_encoded['year_of_birth']

In [ ]:
from tqdm import tqdm
import itertools
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier


In [ ]:
white_no_ecz = whites_encoded.drop(columns = ['Eczema'])
y = whites_encoded['Eczema']

white_column_check = white_no_ecz.columns
white_dtype = white_no_ecz.dtypes.tolist()

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(white_no_ecz) 
# pca = PCA(n_components=0.95)
# X_pca = pca.fit_transform(X_scaled)


X_train, X_test, y_train, y_test = train_test_split(white_no_ecz, y, test_size=0.2, random_state=13)

model = RandomForestClassifier(random_state=95, class_weight = 'balanced')

# model = BaggingClassifier(base_estimator=base_model, 
#                                    n_estimators=10,  # Number of models to bag
#                                    max_samples=0.8,  # Fraction of training data per model
#                                    max_features=0.8,  # Fraction of features per model
#                                    random_state=42,
#                                    bootstrap=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

In [ ]:
import seaborn as sns
plt.figure()
sns.heatmap(conf_matrix, annot=True, cmap="Reds", fmt="d", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('White Population Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
feature_importances = model.feature_importances_

# Get the feature names
feature_names = X_train.columns

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df = feature_importance_df.drop(1)
top_features = feature_importance_df.head(20)

In [ ]:
del white_only, whites_encoded, X_train, X_test, y_train, y_test, cat_columns
gc.collect()

In [ ]:
top_white = white_no_ecz[top_features['Feature'].tolist()]
del white_no_ecz
gc.collect()

In [ ]:
X_train_top, X_test_top, y_train_top, y_test_top = train_test_split(top_white, y, test_size=0.2, random_state=13)
model_top = RandomForestClassifier(random_state=95, class_weight = 'balanced')

model_top.fit(X_train_top, y_train_top)
y_pred_top = model_top.predict(X_test_top)
cm_top = confusion_matrix(y_test_top, y_pred_top)

accuracy_topw = accuracy_score(y_test_top, y_pred_top)
conf_matrix_topw = confusion_matrix(y_test_top, y_pred_top)

print(f'Accuracy: {accuracy_topw}')
print('Confusion Matrix:')
print(conf_matrix_topw)

In [ ]:
plt.figure()
sns.heatmap(conf_matrix_topw, annot=True, cmap="Reds", fmt="d", xticklabels=np.unique(y_test_top), yticklabels=np.unique(y_test_top))
plt.title('White Population Confusion Matrix (Top Features)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
del X_train_top, X_test_top, y_train_top, y_test_top

In [ ]:
blacks_only = final_df[final_df['race'] == 'Black or African American']
blacks_only.loc[:, 'year_of_birth'] = blacks_only['date_of_birth'].dt.year
blacks_only = blacks_only.drop(['person_id', 'race', 'race_concept_id', 'gender_concept_id', 'ethnicity_concept_id','sex_at_birth_concept_id', 'observation_datetime', 'date_of_birth'], axis = 1)
cat_columns_blacks = blacks_only.select_dtypes(include=['object', 'category']).columns
blacks_encoded = pd.get_dummies(blacks_only, columns=cat_columns_blacks, drop_first=True)
X_blacks = blacks_encoded.drop(columns = ['Eczema'])
y_blacks = blacks_encoded['Eczema']

del blacks_only, blacks_encoded, cat_columns_blacks, final_df
gc.collect()

In [ ]:
column_diff = set(white_column_check) - set(X_blacks.columns)
column_diff = list(column_diff)
blacks_dict = {column_diff[i]: 'unknown' if white_dtype[i] == object else 0 for i in range(len(column_diff))}
add_df_blacks = pd.DataFrame(blacks_dict, index=X_blacks.index)

X_blacks_full = pd.concat([X_blacks, add_df_blacks], axis=1)
    
X_blacks_full = X_blacks_full[white_column_check]
X_blacks_full

In [ ]:
y_pred_blacks = model.predict(X_blacks_full)
accuracy_blacks = accuracy_score(y_blacks, y_pred_blacks)
conf_matrix_blacks = confusion_matrix(y_blacks, y_pred_blacks)

print(f'Accuracy: {accuracy_blacks}')
print('Confusion Matrix:')
print(conf_matrix_blacks)

In [ ]:
plt.figure()
sns.heatmap(conf_matrix_blacks, annot=True, cmap="Reds", fmt="d", xticklabels=np.unique(y_blacks), yticklabels=np.unique(y_blacks))
plt.title('Black Population Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
top_black = X_blacks[top_features['Feature'].tolist()]

In [ ]:
y_pred_blacks_top = model_top.predict(top_black)
accuracy_topb = accuracy_score(y_blacks, y_pred_blacks_top)
conf_matrix_blacks_topb = confusion_matrix(y_blacks, y_pred_blacks_top)

print(f'Accuracy: {accuracy_topb}')
print('Confusion Matrix:')
print(conf_matrix_blacks_topb)

In [ ]:
plt.figure()
sns.heatmap(conf_matrix_blacks_topb, annot=True, cmap="Reds", fmt="d", xticklabels=np.unique(y_blacks), yticklabels=np.unique(y_blacks))
plt.title('Black Population Confusion Matrix (Top Features)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
top_features.to_csv('top_features.csv', index=False)

In [ ]:
from IPython.display import FileLink

# Create a download link
FileLink(r'top_features.csv')